In [2]:
# custom_csv_detection_dataset.py
import os
import pandas as pd
import torch
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as T

class CustomCSVDetectionDataset(Dataset):
    def __init__(self, csv_file, images_dir, transforms=None):
        self.df = pd.read_csv(csv_file)
        self.images_dir = images_dir
        self.transforms = transforms
        self.image_ids = self.df['image_name'].unique()

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        records = self.df[self.df['image_name'] == image_id]

        image = Image.open(os.path.join(self.images_dir, image_id)).convert("RGB")
        boxes = []
        labels = []

        for _, row in records.iterrows():
            xmin = row['bbox_x']
            ymin = row['bbox_y']
            xmax = xmin + row['bbox_width']
            ymax = ymin + row['bbox_height']
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(1 if row['label_name'] == 'Marine Animal' else 2)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image_id = torch.tensor([idx])

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id

        # Convert image to tensor if no transformations are applied
        if self.transforms:
            image = self.transforms(image)
        else:
            image = T.ToTensor()(image)

        return image, target

In [3]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: F:\sml_project
